In [39]:
import sqlite3
import pyodbc
import csv
import pandas as pd
import datetime
from azure.storage.blob import BlobClient

StatementMeta(notebookrun, 19, 39, Finished, Available)

In [40]:
server = ''
database = ''
username = ''
password = ''
driver = ''
blob_file_path = ''
execution_id_path = ''
blob_account_name = ''
blob_account_key = ''
container = ''
SSISProject = ''

StatementMeta(notebookrun, 19, 40, Finished, Available)

In [41]:
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import os

app_name = 'SSISDB_Get_Params'
my_jars = os.environ.get("SPARK_HOME")
myconf = SparkConf()
myconf.setMaster("local").setAppName("SSISDB_Get_Params")
myconf.set("spark.jars","%s/jars/log4j-1.2.17.jar" % my_jars)
spark = SparkSession\
 .builder\
 .appName("SSISDB_Get_Params")\
 .config(conf = myconf) \
 .getOrCreate()


Logger= spark._jvm.org.apache.log4j.Logger
mylogger = Logger.getLogger(app_name)

StatementMeta(notebookrun, 19, 41, Finished, Available)

In [42]:
def log_msgs(msg_type,msg):
        
        if msg_type.upper() == "ERROR":
            print('ERROR: %s' % repr(msg))
            mylogger.error(repr(msg))
        else:
            print('INFO: %s' % repr(msg))
            mylogger.info(repr(msg))

StatementMeta(notebookrun, 19, 42, Finished, Available)

In [43]:
log_msgs("INFO",'Entities Created/Updated')
log_msgs("INFO", server)
log_msgs("INFO",database)
log_msgs("INFO",username)
log_msgs("INFO",password)
log_msgs("INFO",driver)
log_msgs("INFO",blob_file_path)
log_msgs("INFO",execution_id_path)
log_msgs("INFO",blob_account_name)
log_msgs("INFO",blob_account_key)
log_msgs("INFO",container)
log_msgs("INFO",SSISProject)
log_msgs("INFO",'DRIVER='+f'{driver}'+';SERVER='+f'{server}'+';DATABASE='+f'{database}'+';UID='+f'{username}'+';PWD='+ f'{password}')


StatementMeta(notebookrun, 19, 43, Finished, Available)

INFO: 'Entities Created/Updated'
INFO: ''
INFO: ''
INFO: ''
INFO: ''
INFO: ''
INFO: ''
INFO: ''
INFO: ''
INFO: ''
INFO: ''
INFO: ''
INFO: 'DRIVER=;SERVER=;DATABASE=;UID=;PWD='

In [44]:
blob_connection_string = 'DefaultEndpointsProtocol=https;AccountName=' + blob_account_name + ';AccountKey=' + blob_account_key + \
';EndpointSuffix=core.windows.net'

execution_id_url = 'abfss://' + container + '@' + blob_account_name + '.dfs.core.windows.net/' + execution_id_path

PARAM_Query = \
"SELECT CAST(a.[parameter_name] as nvarchar(max)) as parameter_name,CAST(a.parameter_value as nvarchar(max)) as parameter_value  \
FROM [SSISDB].[internal].[execution_parameter_values] a \
INNER JOIN [SSISDB].[internal].[executions] b \
ON ( a.execution_id = b.execution_id ) \
WHERE [object_type] = 30 and a.[execution_id] = \
(SELECT max([execution_id]) \
from [SSISDB].[internal].[execution_parameter_values]) \
AND a.[parameter_value] IS NOT NULL \
AND a.[parameter_value] != 0 \
AND b.[project_name] = '" + f'{SSISProject}' + "' \
GROUP BY a.[object_type],a.[parameter_name],a.[parameter_value],b.[project_name],a.execution_id"

connection = pyodbc.connect('DRIVER='+f'{driver}'+';SERVER='+f'{server}'+';DATABASE='+f'{database}'+';UID='+f'{username}'+';PWD='+ f'{password}')

StatementMeta(notebookrun, 19, 44, Finished, Cancelled)

In [45]:
Get_execution_id = "SELECT CAST(MAX(a.execution_id) as int) as execution_id FROM [SSISDB].[internal].[execution_parameter_values] a"
cursor = connection.cursor()
cursor.execute(Get_execution_id)
rows_check = cursor.fetchall()
columns_check = [column[0] for column in cursor.description]
if rows_check[0][0] is None:
    New_Execution_Id = 0
else:
    New_Execution_Id = int(rows_check[0][0]) 

StatementMeta(notebookrun, 19, 45, Finished, Available)

In [46]:
Old_Execution_Id = None
if New_Execution_Id != 0:
    try:
        df = spark.read.load(execution_id_url, format='csv', header=True).toPandas()
        for index, row in df.iterrows():
            row.execution_id
            Old_Execution_Id = int(row.execution_id)
    except:
        pass
else:
    df_Execution_Id = pd.DataFrame.from_records(rows_check, columns=columns_check)
    print(df_Execution_Id)
    if df_Execution_Id['execution_id'][0] is None:
        Old_Execution_Id = None
        New_Execution_Id = None
    else:
        df_sp_df_Execution_Id = spark.createDataFrame(df_Execution_Id)

        df_sp_df_Execution_Id.write.option('header', 'true').mode('overwrite').csv(execution_id_url)
        Old_Execution_Id = 0
if (Old_Execution_Id is None and New_Execution_Id is not None) or (Old_Execution_Id < New_Execution_Id):
    connection = pyodbc.connect('DRIVER='+f'{driver}'+';SERVER='+f'{server}'+';DATABASE='+f'{database}'+';UID='+f'{username}'+';PWD='+ f'{password}')
    cursor = connection.cursor()
    # Execute the query
    cursor.execute(PARAM_Query)
    rows = cursor.fetchall()
    # # get column names from cursor
    columns = [column[0] for column in cursor.description]
    df_Params = pd.DataFrame.from_records(rows, columns=columns)
    csvdata = df_Params.to_csv('Params.txt')
    blob = BlobClient.from_connection_string(conn_str=f'{blob_connection_string}', container_name=f'{container}', blob_name=f'{blob_file_path}')
    with open("Params.txt", "rb") as data:
       blob.upload_blob(data,overwrite=True)
   
    df_Execution_Id = pd.DataFrame.from_records(rows_check, columns=columns_check)
    df_sp_df_Execution_Id = spark.createDataFrame(df_Execution_Id)
    df_sp_df_Execution_Id.write.option('header', 'true').mode('overwrite').csv(execution_id_url)
    mssparkutils.notebook.exit(True) 
else: 
    mssparkutils.notebook.exit(False)  


StatementMeta(notebookrun, 19, 46, Finished, Available)

OperationalError: ('HYT00', '[HYT00] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0) (SQLDriverConnect)')